# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet'])

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, confusion_matrix

[nltk_data] Downloading package punkt to /home/freemo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/freemo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("Disaster_Data", engine)
X = df['message']
#drop also "related" column as there were records with 3 choices
y = df.drop(['id', 'message', 'original', 'genre', 'related'], axis = 1)
categories = y.columns

In [3]:
categories

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [4]:
# check df
df.to_csv("df.csv", index=True)

In [5]:
# check the shape
X.shape
y.shape

(26216, 35)

Check randomly the two arrays

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """Tokenize and clean text.
    
    Args: message text
    
    Returns: normalized tokens
    """
    # remove punctuation
    text = re.sub(r'[^a-zA-Z0-9]', ' ',text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [7]:
# test function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/

In [39]:
%timeit

In [27]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=0, n_jobs=-1, class_weight="balanced")))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
# split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [29]:
# train classifier, around 8 mins to complete / it took just 3mins with the new settings
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f96da474a70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        n_jobs=-1,
                                                                        random_state=0)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [30]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [31]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5459
           1       0.85      0.47      0.61      1095

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.77      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

------------------------------------------------------

FEATURE: aid_related

              precision    recall  f1-score   support

           0       0.77      0.87      0.82      3829
           1 

/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5932
           1       0.77      0.25      0.37       622

    accuracy                           0.92      6554
   macro avg       0.85      0.62      0.67      6554
weighted avg       0.91      0.92      0.90      6554

------------------------------------------------------

FEATURE: fire

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6489
           1       0.00      0.00      0.00        65

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

------------------------------------------------------

FEATURE: earthquake

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5943
           1       0.92      0.50      0.65       611

    accuracy                      

In [32]:
# display results

# Print classification report on test data
print(classification_report(y_test, y_pred, target_names=y.columns))

                        precision    recall  f1-score   support

               request       0.85      0.47      0.61      1095
                 offer       0.00      0.00      0.00        30
           aid_related       0.78      0.64      0.70      2725
          medical_help       0.71      0.04      0.07       517
      medical_products       0.86      0.05      0.10       339
     search_and_rescue       0.50      0.01      0.01       186
              security       0.00      0.00      0.00       141
              military       0.00      0.00      0.00       208
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.17      0.29       413
                  food       0.90      0.31      0.46       725
               shelter       0.88      0.16      0.27       599
              clothing       0.56      0.05      0.09       103
                 money       0.75      0.02      0.04       143
        missing_people       0.00      

/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [33]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [34]:
# check all features
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f96da474a70>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                          n_jobs=-1,
                                                          random_state=0)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f96da474a70>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                        n_jobs=-1,
                                                        random_state=0)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 '


parameters = {'tfidf__norm': ['l1','l2'],
              'clf__estimator__criterion': ["gini", "entropy"]
    
             }

In [40]:
# working model
# parameters = {'clf__estimator__n_estimators': [100, 200]}

# rf_Grid.best_params_

In [41]:
parameters = {'tfidf__norm': ['l1','l2'],
              'clf__estimator__criterion': ["gini", "entropy"]
    
             }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [42]:
# nothing for 15min
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f96da474a70>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                                               n_jobs=-1,
                                                                                               random_state=0)))]),
             param_grid={'clf__estimator__criterion': ['gini', 'entropy'],
                         'tfidf__norm': ['l1', 'l2']})

In [44]:
cv.best_params_

{'clf__estimator__criterion': 'entropy', 'tfidf__norm': 'l2'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
y_pred = cv.predict(X_test)

In [46]:
print(classification_report(y_test, y_pred, target_names=y.columns))

                        precision    recall  f1-score   support

               request       0.87      0.46      0.60      1095
                 offer       0.00      0.00      0.00        30
           aid_related       0.79      0.62      0.70      2725
          medical_help       0.78      0.05      0.09       517
      medical_products       0.81      0.04      0.07       339
     search_and_rescue       0.00      0.00      0.00       186
              security       0.00      0.00      0.00       141
              military       0.00      0.00      0.00       208
           child_alone       0.00      0.00      0.00         0
                 water       0.94      0.15      0.27       413
                  food       0.88      0.31      0.46       725
               shelter       0.90      0.15      0.26       599
              clothing       0.64      0.07      0.12       103
                 money       0.75      0.02      0.04       143
        missing_people       1.00      

/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/freemo/anaconda3/lib/python3.7/si

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.